In [1]:
import psycopg2

In [12]:
# Update connection string information obtained from the portal
host = "wangandyoondatabase.postgres.database.azure.com"
user = "mylogin@wangandyoondatabase"
dbname = "postgres"
password = "yourpassword"
sslmode = "require"

In [3]:
# Construct connection string
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
conn = psycopg2.connect(conn_string) 
print("Connection established")

Connection established


In [7]:
cursor=conn.cursor()


In [5]:
import psycopg2
# Update connection string information obtained from the portal
host = "wangandyoondatabase.postgres.database.azure.com"
user = "mylogin@wangandyoondatabase"
dbname = "postgres"
password = "shampoo1!"
sslmode = "require"

# Construct connection string
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
conn = psycopg2.connect(conn_string)
print("Connection established")

cursor=conn.cursor()

cursor.execute("CREATE TABLE SentimentAnalysisTable (id serial PRIMARY KEY, UserID VARCHAR(50), Sentiment VARCHAR(10), Date VARCHAR(50) );")
print("Finished creating table")

from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import time
import json
from nltk.sentiment.vader import SentimentIntensityAnalyzer as Vader

# consumer key, consumer secret, access token, access secret.
ckey = "DWP7ziCeLmIF7EM6E3U1x4Awk"
csecret = "8AbR2jazy2FBNMu8d4Yyma6YPieSaEZniNBvCWngaN40BvTAO9"
atoken = "959357240787349505-xwLcOZ5LoVMXsLwlVzHcx3eHAOyB6SS"
asecret = "XfXLhs7dC7tHb49PmfbTkSFLoWBYoFmInFC6uMG5aLalS"


class listener(StreamListener):

    def on_data(self, data):
        Data = json.loads(data)

        if 'RT @' not in Data['text']:

            tweet = Data["text"]
            ID=Data["user"]["name"]
            Date=Data["created_at"]
            sid = Vader()
            ss = sid.polarity_scores(tweet)
            if ss['pos'] > 0 and ss['compound'] > 0:
                sentimentalStatus = 'Positive'
            elif ss['neg'] > 0 and ss['compound'] < 0:
                sentimentalStatus = 'Negative'
            else:
                sentimentalStatus = 'Neutural'
            cursor.execute("INSERT INTO SentimentAnalysisTable (UserID, Sentiment, Date) VALUES (%s, %s, %s);",
                           (ID, sentimentalStatus, Date))
            print("Inserted 1 row of data")
            print(tweet + " Sentiment Satus:" + sentimentalStatus)
            output = open("finalcoinSentiment.csv", "a", encoding='utf-8')
            output.write(Date+",")
            output.write(ID+",")
            output.write(sentimentalStatus+"\n")
            output.close()
            time.sleep(1)
            return True

    def on_error(self, status):
        print(status)


auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["Bitcoin","Cryptocrunccy","Ethereum","Ripple","Blockchain","Tether"])



Connection established


ProgrammingError: relation "sentimentanalysistable" already exists


In [9]:
# Fetch all rows from table
cursor.execute("SELECT * FROM SentimentAnalysisTable;")
rows = cursor.fetchall()

InternalError: current transaction is aborted, commands ignored until end of transaction block


In [10]:
# Print all rows
print(len(rows))
for row in rows:
    print(row[3])
    print("Data row = (%s, %s, %s, %s)" %(str(row[0]), str(row[1]), str(row[2]), str(row[3])))

NameError: name 'rows' is not defined

In [11]:
# Cleanup
conn.commit()
cursor.close()
conn.close()